# Searching for faster matrix multiplication algorithms with quantum annealing

In [1]:
import dimod
import pennylane as qml
from pennylane import qaoa
from pennylane import numpy as np
from matplotlib import pyplot as plt
from utils import *
from optimized_qubos import *
from ocean_pennylane_integration import *

In [2]:
dim = 2
initial_tensor = get_standard_tensor(dim) % 2
origo = np.tensordot([0]*4, np.tensordot([0]*4, [0]*4, axes=0), axes=0)

In [3]:
strassen_tensors = [np.tensordot([0,0,0,1], np.tensordot([-1,0,1,0], [1,0,1,0], axes=0), axes=0),
          np.tensordot([1,1,0,0], np.tensordot([0,0,0,1], [-1,1,0,0], axes=0), axes=0),
           np.tensordot([-1,0,1,0], np.tensordot([1,1,0,0], [0,0,0,1], axes=0), axes=0),
           np.tensordot([1,0,0,1], np.tensordot([1,0,0,1], [1,0,0,1], axes=0), axes=0),
          np.tensordot([0,1,0,-1], np.tensordot([0,0,1,1], [1,0,0,0], axes=0), axes=0),
           np.tensordot([1,0,0,0], np.tensordot([0,1,0,-1], [0,1,0,1], axes=0), axes=0),
           np.tensordot([0,0,1,1], np.tensordot([1,0,0,0], [0,0,1,-1], axes=0), axes=0)]

In [4]:
tensor = (initial_tensor - strassen_tensors[0] - strassen_tensors[1] - strassen_tensors[2] - strassen_tensors[3]) % 2

# Moving towards origo
while(True):
    bqm = towards_user_defined_small(tensor, origo, dim)
    params, qaoa_circuit, wires = construct_qaoa_and_optimize(bqm)
    print_probs(qaoa_circuit, wires, params)
    #sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
    #print(sampleset)
    #x1, y1, z1 = process_result(sample, 2)
    print(x1, y1, z1)
    tensor = (tensor - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
    if np.count_nonzero(tensor.flatten()) == 0:
        print("End")
        break
        
tensor = (initial_tensor + strassen_tensors[0] + strassen_tensors[1] + strassen_tensors[2]) % 2
# Moving towards standard matrix multiplication i.e. the naive method
while(True):
    bqm = towards_user_defined_small(tensor, initial_tensor, dim)
    #sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
    #print(sampleset)
    #x1, y1, z1 = process_result(sample, 2)
    #print(x1, y1, z1)
    tensor = (tensor - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
    if np.array_equal(tensor, initial_tensor):
        print("End")
        break

Cost Hamiltonian:   (0.0) [Z0]
+ (0.0) [Z1]
+ (0.0) [Z3]
+ (0.0) [Z4]
+ (0.0) [Z6]
+ (0.0) [Z7]
+ (0.0) [Z9]
+ (0.0) [Z10]
+ (0.0) [Z12]
+ (0.0) [Z14]
+ (0.0) [Z16]
+ (0.0) [Z18]
+ (0.0) [I0]
+ (36.0) [Z2]
+ (36.0) [Z5]
+ (36.0) [Z8]
+ (36.0) [Z11]
+ (36.0) [Z13]
+ (36.0) [Z15]
+ (36.0) [Z17]
+ (36.0) [Z19]
+ (36.0) [Z20]
+ (36.0) [Z21]
+ (36.0) [Z22]
+ (36.0) [Z23]
+ (36.0) [Z24]
+ (36.0) [Z25]
+ (36.0) [Z26]
+ (36.0) [Z27]
+ (36.0) [Z28]
+ (-24.0) [Z2 Z0]
+ (-24.0) [Z2 Z1]
+ (-24.0) [Z5 Z3]
+ (-24.0) [Z5 Z4]
+ (-24.0) [Z8 Z6]
+ (-24.0) [Z8 Z7]
+ (-24.0) [Z11 Z9]
+ (-24.0) [Z11 Z10]
+ (-24.0) [Z13 Z9]
+ (-24.0) [Z13 Z12]
+ (-24.0) [Z15 Z10]
+ (-24.0) [Z15 Z14]
+ (-24.0) [Z17 Z9]
+ (-24.0) [Z17 Z16]
+ (-24.0) [Z19 Z1]
+ (-24.0) [Z19 Z18]
+ (-24.0) [Z20 Z1]
+ (-24.0) [Z20 Z4]
+ (-24.0) [Z21 Z0]
+ (-24.0) [Z21 Z3]
+ (-24.0) [Z22 Z12]
+ (-24.0) [Z22 Z14]
+ (-24.0) [Z23 Z14]
+ (-24.0) [Z23 Z16]
+ (-24.0) [Z24 Z3]
+ (-24.0) [Z24 Z18]
+ (-24.0) [Z25 Z1]
+ (-24.0) [Z25 Z6]
+ (-24.0) [Z26 Z7]


MemoryError: Unable to allocate 8.00 GiB for an array with shape (2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2) and data type complex128